In [1046]:
import pandas as pd 
from datetime import  datetime , timedelta 

- # Load Data 

In [1047]:
df = pd.read_csv(r"C:\Users\Alaa\Desktop\DataInternEu\linkedin_internships25.csv")
df


,job_id,job_title,company_name,company_location,posted_time
0,4204952998,Business & Data Analyst,Karo Healthcare,"Stockholm, Stockholm County, Sweden",3 weeks ago
1,4205941527,Data Analyst,EpicFrog,"Riga, Riga, Latvia",1 day ago
2,4206694302,Data Scientist,Unilever,"Wageningen, Gelderland, Netherlands",1 day ago
3,4204887150,Data Science Apprentice,Gaming Innovation Group,"San Ġiljan, Saint Julian's, Malta",2 days ago
4,4204705711,Quant / Data Scientist,Venus Labs,Greater Dublin,3 days ago
...,...,...,...,...,...
350,4204747737,2x Data Engineer Interns,Steelcase,"Cluj-Napoca, Cluj, Romania",3 days ago
351,4206805447,"BA Credit Risk & Compliance (100% remote, only...",UST España & Latam,Spain,21 hours ago
352,4206034224,Artificial Intelligence Engineer,Ubique Systems,"Barcelona, Catalonia, Spain",19 hours ago
353,4203322808,Data engineer / Machine Learning engineer,Zirak,Italy,3 days ago


# 1. Clean location: Split into city, country (ignore region)


In [1048]:
df['country'] = df['company_location'].str.split(',').str.get(2)
df['city'] = df['company_location'].str.split(',').str[:2].str.join(' ')
df.isna().sum()

job_id               0
job_title            5
company_name         5
company_location     5
posted_time          5
country             76
city                 5
dtype: int64

In [1049]:
df.head(5)

,job_id,job_title,company_name,company_location,posted_time,country,city
0,4204952998,Business & Data Analyst,Karo Healthcare,"Stockholm, Stockholm County, Sweden",3 weeks ago,Sweden,Stockholm Stockholm County
1,4205941527,Data Analyst,EpicFrog,"Riga, Riga, Latvia",1 day ago,Latvia,Riga Riga
2,4206694302,Data Scientist,Unilever,"Wageningen, Gelderland, Netherlands",1 day ago,Netherlands,Wageningen Gelderland
3,4204887150,Data Science Apprentice,Gaming Innovation Group,"San Ġiljan, Saint Julian's, Malta",2 days ago,Malta,San Ġiljan Saint Julian's
4,4204705711,Quant / Data Scientist,Venus Labs,Greater Dublin,3 days ago,NaN,Greater Dublin


# 2. Handle missing Values

In [1050]:
df["city"] = df["city"].fillna("Unknown City")
df['country'] = df['country'].fillna('Unknown Country')
df = df.drop(columns=['posted_time', 'company_location','job_id'])
df = df.dropna()
df.isna().sum()


job_title       0
company_name    0
country         0
city            0
dtype: int64

In [1051]:
print(df.head(10))


                                  job_title             company_name  \
0                   Business & Data Analyst          Karo Healthcare   
1                              Data Analyst                 EpicFrog   
2                            Data Scientist                 Unilever   
3                   Data Science Apprentice  Gaming Innovation Group   
4                    Quant / Data Scientist               Venus Labs   
5                       Carbon Data Analyst              Fastmarkets   
6                    Data Analyst - Product               TravelPerk   
7          Internship - Data Science (HIVE)                 Unilever   
8  Financial Planning & Analysis Controller              Avis Greece   
9     ML Eng./Data Science Python Developer                TST Group   

           country                         city  
0           Sweden  Stockholm  Stockholm County  
1           Latvia                   Riga  Riga  
2      Netherlands       Wageningen  Gelderland  
3      

In [1052]:
df['country'].value_counts()

country
Unknown Country    71
 France            46
 Spain             35
 Germany           29
 Italy             26
 Portugal          21
 Poland            21
 Netherlands       20
 Ireland           13
 Belgium           12
 Greece             8
 Denmark            7
 Romania            7
 Bulgaria           6
 Czechia            5
 Sweden             5
 Hungary            4
 Austria            3
 Lithuania          2
 Latvia             2
 Malta              2
 Cyprus             2
 Luxembourg         1
 Croatia            1
 Slovakia           1
Name: count, dtype: int64

# 3.Define Job Types

In [1053]:
JOB_TYPE_KEYWORDS = {
    # Data Engineers (EN/FR)
    "data engineer": [
        # English
        "engineer", "etl", "pipeline", "data pipeline", "infrastructure",
        "warehouse", "big data", "spark", "hadoop", "kafka", "airflow",
        "data ops", "dataops", "database", "data model", "data lake",
        "cloud", "aws", "azure", "gcp", "snowflake", "databricks",
        # French
        "ingénieur", "etl", "pipeline", "infrastructure données",
        "entrepôt données", "big data", "spark", "hadoop", "kafka",
        "dataops", "base de données", "modèle données", "lac données",
        "cloud", "ingénieur données", "ingénieur cloud"
    ],
    
    # Data Analysts (EN/FR)
    "data analyst": [
        # English
        "analyst", "analytics", "bi", "business intelligence", "reporting", 'analist',
        "tableau", "powerbi", "power bi", "looker", "visualization", "dashboard",
        "kpi", "metrics", "excel", "sql", "statistics", "business analyst",
        "product analyst", "financial analyst", "marketing analyst", "risk analyst",
        "commercial analyst", "operations analyst", "retail media", "quality expert",
        # French
        "analyste", "analytics", "bi", "intelligence d'affaires", "reporting",
        "visualisation", "tableau de bord", "indicateurs", "métriques",
        "analyste financier", "analyste marketing", "analyste produit",
        "analyste risque", "analyste commercial", "analyste opérationnel",
        "expert qualité", "analyste données cliniques"
    ],

    # Data Scientists (EN/FR)
    "data scientist": [
        # English
        "scientist", "machine learning", "ml", "ai", "artificial intelligence",
        "research", "deep learning", "dl", "nlp", "computer vision", "predictive",
        "modeling", "quantitative", "ai/ml", "ai engineer", "ml engineer",
        "applied scientist", "data science", "llm", "gen ai", "computational",
        "research engineer", "r&d", "energy data", "security field",
        # French
        "scientifique", "apprentissage automatique", "ml", "ia", "intelligence artificielle",
        "recherche", "deep learning", "dl", "traitement langage", "vision par ordinateur",
        "prédictif", "modélisation", "quantitatif", "ia/ml", "ingénieur ia",
        "ingénieur ml", "science des données", "recherche et développement",
        "données énergie", "sécurité informatique", "scientifique données"
    ],
    
    # Business Intelligence (EN/FR)
    "business intelligence": [
        # English
        "bi ", "business intelligence", "bi developer", "bi specialist",
        "bi analyst", "bi consultant", "bi tools",
        # French
        "bi ", "intelligence d'affaires", "développeur bi", "spécialiste bi",
        "analyste bi", "consultant bi", "outils bi"
    ],

    # Data Product & Management (EN/FR)
    "data product": [
        # English
        "product", "data product", "strategy", "governance", "management",
        "architect", "chief data", "director of data", "head of data",
        "data governance", "data strategy",
        # French
        "produit", "produit données", "stratégie", "gouvernance", "gestion",
        "architecte", "directeur données", "responsable données",
        "gouvernance données", "stratégie données"
    ],

    # Catch-All (Fallback)
    "other": []
}

# 4.Categorize Job Titles

In [1054]:
def categorize_job(title: str) -> str:
    title = title.lower()  
    for job_type, keywords in JOB_TYPE_KEYWORDS.items():
        if any(keyword in title for keyword in keywords):
            return job_type
    return "other"  

df["job_type"] = df["job_title"].apply(categorize_job)


In [1055]:
df['job_type'].value_counts()

job_type
data analyst      141
data scientist     90
other              52
data engineer      49
data product       18
Name: count, dtype: int64

In [1056]:
df.head(30)

,job_title,company_name,country,city,job_type
0,Business & Data Analyst,Karo Healthcare,Sweden,Stockholm Stockholm County,data analyst
1,Data Analyst,EpicFrog,Latvia,Riga Riga,data analyst
2,Data Scientist,Unilever,Netherlands,Wageningen Gelderland,data scientist
3,Data Science Apprentice,Gaming Innovation Group,Malta,San Ġiljan Saint Julian's,data scientist
4,Quant / Data Scientist,Venus Labs,Unknown Country,Greater Dublin,data scientist
5,Carbon Data Analyst,Fastmarkets,Bulgaria,Sofia Sofia City,data analyst
6,Data Analyst - Product,TravelPerk,Spain,Barcelona Catalonia,data analyst
7,Internship - Data Science (HIVE),Unilever,Netherlands,Wageningen Gelderland,data scientist
8,Financial Planning & Analysis Controller,Avis Greece,Greece,Khalándrion Attiki,data scientist
9,ML Eng./Data Science Python Developer,TST Group,Spain,Barcelona Catalonia,data scientist


In [1057]:
df.to_csv("cleaned_data_internships_eu.csv", index=False)